# Setup

### additional dependencies/configuration

In [ ]:
!pip install geopandas --user -q

### modules

In [1]:
from functools import partial
import pyproj
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, shape
from shapely.ops import transform
import geopyspark as gps
import geopandas as gpd
import fiona

In [2]:
from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [3]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '1200M')
sc = SparkContext(conf = conf)

### create overlay context

In [4]:
hou = an.Overlay('houston')

In [5]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [6]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

### flood layer

In [7]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/flood/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_flood_category5.tif'
mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

In [8]:
hou.layerset_from_tiff(maskedtiff, 'flood')

In [9]:
M.add_layer(hou.layersets['flood'].mappable_layer)

### sea level rise

In [10]:
# download and unzip sea level rise polygons
u = "https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/LA/LA_3_slr_data_dist.zip"
d = "/home/hadoop/notebooks/data/write/overlay-layers/sea-level-rise/"
# data.zip_to_dir(u, d)

In [11]:
# locate .gdb file
gdb = "/home/hadoop/notebooks/data/write/overlay-layers/sea-level-rise/LA_3_slr_data_dist/LA_3_slr_final_dist_polys.gdb/"

In [12]:
tt = data.read_from_gdb(gdb, 1, MultiPolygon)

In [ ]:
hou.layerset_from_poly(tt, 1, 'sea_level', 'plasma', 1)